In [89]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
# Attaching 3D axis to the figure
import os

#preprocess
def get_video(src_file):
    video = src_file.read().splitlines()
    # video[:,3,6,9...]
    for i in range(len(video)):
        frame = video[i].split(',')[1:]#escape the first number
        frame = [float(j) for j in frame] # turn them into float
        video[i] = frame
        
    video = np.array(video)
    for i in range(video.shape[1]/3):
        video[:,i+2] = video[:,i+2]/7.8125
    return video

def get_subject_video(video,subject):
    return video[:,subject*45:subject*45+45]


def get_act_subject(video):
    subject_dist = list()
    for subject in [0,1]:
        total_dist = 0
        s_video = get_subject_video(video,subject)
        for i in range(1,len(video)):
            # print i
            total_dist = total_dist + vect_length(s_video[i,:]-s_video[i-1,:])
        subject_dist.append(total_dist)    
    if subject_dist[0]>subject_dist[1]:
        act_subject = 0
    else :
        act_subject = 1
    return act_subject


def divide_video(video):
    act_subject = get_act_subject(video)
    react_subject = 1-act_subject
    act_video = get_subject_video(video, act_subject)
    react_video = get_subject_video(video, react_subject)
    return act_video, react_video

def vect_length(vect):
    return np.dot(vect,vect)**0.5

def dist_from_joints_to_joint(joints,joint):
    # assume joints is (x1,y1,z1,x2,y2,z2,...)
    joints_num = len(joints)/3
    
    dist_list = np.zeros(joints_num)
    for i in range(joints_num):
        vect = get_joint(joints,i)-joint
        dist_list[i] = vect_length(vect)
    return dist_list

def pos_from_joints_to_joint(joints,joint):
    temp = np.array([])
    for i in range(len(joints)/3):
        temp = np.append(temp,joint)
    return joints-temp

def dist_from_video_to_joint(video,joint):
    dist_mat = np.array([])
    for i in range(len(video)):
        dist_list = dist_from_joints_to_joint(video[i,:], joint)
        dist_mat = np.vstack((dist_mat, dist_list)) if dist_mat.size else dist_list
    return dist_mat

def get_joint(video,joint):
    if len(video.shape)==1: #only a frame
        return video[3*joint:3*joint+3]
    else:
        return video[:, 3*joint:3*joint+3]


path = 'sbu_cleaned/01/s01s02001.txt'
with open(path,'r') as src_file:
    video = get_video(src_file)
    print video.shape
    act_video, react_video = divide_video(video)
    
    joint = get_joint(react_video[1,:],5)
    joints = react_video[1,:]
    
    print dist_from_joints_to_joint(joints, joint)
    
    print dist_from_video_to_joint(act_video,joint).shape
    
    print pos_from_joints_to_joint(joints, joint)

(45, 90)
[ 0.29323264  0.04613507  0.03517929  0.04042785  0.01994394  0.
  0.05581112  0.05795737  0.0562305   0.01881537  1.48317494  1.827599
  1.50379333  1.49925482  1.53546079]
(45, 15)
[ 0.18194976  0.22943016 -0.0155392  -0.00513664 -0.041792   -0.0188544
 -0.00543104 -0.02668416 -0.022272   -0.00312064 -0.0401664  -0.0033664
 -0.00354304 -0.01925248  0.0038144   0.          0.          0.
 -0.00715264 -0.04341888 -0.0343296  -0.0113152  -0.02518784 -0.0509568
 -0.00731648 -0.00615936 -0.0554112  -0.0044416  -0.01056128 -0.0149248
 -0.00275328  0.01945344  1.4830448   0.19873976  1.04938016  1.4830448
  0.16521976  0.56901016  1.3821448   0.16594976  0.79372016  1.2610448
  0.16664976  1.00978016  1.1446448 ]


In [81]:
# DONE Procesing in each directory of action class
# Find a subject as the first action

ALLOWED_EXTENSIONS=set(['txt'])

def allowed_files(filename):
    return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXTENSIONS



TORSO = 2


def joint_intent_extract(path, joint, subact_length, show_fag=False):
    with open(path,'r') as src_file:
        video = get_video(src_file)
        act_video, react_video = divide_video(video)
        
        # torso positions
        act_torso = get_joint(act_video[0,:], TORSO)
        react_torso = get_joint(react_video[0,:],TORSO)

        act_joint_video = get_joint(act_video, joint)
        react_joint_video = get_joint(react_video, joint)
        # compare the target_points with the dist 
        
        
        act_to_react_torso = dist_from_video_to_joint(act_joint_video,react_torso)
        react_to_act_torso = dist_from_video_to_joint(react_joint_video,act_torso)
               
        labels = np.array([])   
        for i in range(subact_length,len(video)):
            react_movement = react_to_act_torso[i] - react_to_act_torso[i-subact_length]
            act_movement = act_to_react_torso[i] - act_to_react_torso[i-subact_length]
            
            if react_movement>0:
                # intend to leave
                label = 2
            else:
                # intend to reach
                label = 1
            
            if abs(act_movement)>abs(react_movement*2):
                label = 0 # no intent
            
            labels = np.append(labels,label)

    return labels

def skel_feature_extract(path, subact_length):
    with open(path,'r') as src_file:
        video = get_video(src_file)
        act_video, react_video = divide_video(video)
        
        # torso positions
        act_torso = get_joint(act_video[0,:], TORSO)
        react_torso = get_joint(react_video[0,:],TORSO)
        
        dist_react_torso = dist_from_video_to_joint(act_video, react_torso)
        dist_act_torso = dist_from_video_to_joint(act_video, act_torso)
        
        
        # distance feature
        feat0 = dist_act_torso[subact_length:]
        # distance feature
        feat1 = dist_react_torso[subact_length:]
        
        # distance velocity feature
        feat2 = np.array([])
        for i in range(subact_length,len(video)):
            temp = np.array([])
            for d in range(1,1+subact_length):
                temp = np.append(temp, dist_react_torso[i] - dist_react_torso[i-d])
            feat2 = np.vstack((feat2,temp)) if feat2.size else temp
               
        # distance velocity feature
        feat4 = np.array([])
        for i in range(subact_length,len(video)):
            temp = np.array([])
            for d in range(1,1+subact_length):
                temp = np.append(temp, dist_act_torso[i] - dist_act_torso[i-d])
            feat4 = np.vstack((feat4,temp)) if feat4.size else temp
               
        
        # velocity feature 
        feat3 = np.array([])
        for i in range(subact_length, len(video)):
            temp = np.array([])
            for d in range(1,1+subact_length):
                temp = np.append(temp, act_video[i] - act_video[i-d])
            feat3 = np.vstack((feat3, temp)) if feat3.size else temp
        
        # position feature
        feat5 = np.array([])
        for i in range(subact_length, len(video)):
            temp = act_video
            feat5 = np.vstack()
        
        # vector of each 
        feat = np.hstack((feat0, feat1, feat2, feat3, feat4))
        
    return feat


path = 'sbu_cleaned/01/s01s02002.txt'
labels = joint_intent_extract(path, TORSO, 2)
print labels.shape

feat = skel_feature_extract(path,2)
print feat.shape



(22,)
(22, 180)


In [82]:
def build_data_set(target_folder, joint_ind, subact_length, show_fag = False):
    class_lib = list()
    for root,dirs,files in os.walk(target_folder):
        for f in files:
            path = os.path.join(root,f)
            if allowed_files(path):
                class_lib.append(path)

    tt_labels = np.array([])
    tt_feat = np.array([])
    
    for path in class_lib:
        if show_fag:
            print path,
        
        labels = joint_intent_extract(path,joint_ind, subact_length, show_fag)
        tt_labels = np.append(tt_labels, labels)

        feat = skel_feature_extract(path, subact_length)
        
        tt_feat = np.vstack((tt_feat,feat)) if tt_feat.size else feat
        
    return tt_feat, tt_labels



def divide_data_set(act_dict,react_dict,train_rate=0.8):
    react_dict = react_dict.astype('int')
    train_size = int(act_dict.shape[0]*train_rate)

    selected_indices = np.unique(np.random.choice(act_dict.shape[0],train_size,replace=False))
    
    all_indices = range(act_dict.shape[0])
    rest_indices = set(all_indices).difference(selected_indices)
    
    rest_indices = list(rest_indices)
    selected_indices = list(selected_indices)
    
    training_samples = act_dict[selected_indices]
    training_ground = react_dict[selected_indices]
    test_samples = act_dict[rest_indices]
    test_ground = react_dict[rest_indices]

    return training_samples, training_ground, test_samples, test_ground

    #print "count direction error:", count_error,'/',sum(test_results<>test_ground)

folder = "sbu_cleaned/01/"
feats, labels = build_data_set(folder, joint_ind=2 , subact_length=2)
print feats.shape
print labels.shape



(902, 180)
(902,)


In [83]:
def random_precision(labels):    
    tt_proportion = []
    for i in np.unique(labels):
        proportion = sum(labels==i)/float(len(labels))
        tt_proportion.extend([proportion])
    return max(tt_proportion)

CLEANED_FOLDER = 'sbu_cleaned/'
class_i = '05' # shake hands
class_i = '02' # moving
classes = ['01','02','03','04','05','06','07','08']

#classes = ['08']
for class_i in classes:
    target_folder = os.path.join(CLEANED_FOLDER,class_i)
    act_dict, react_dict = build_data_set(target_folder,joint_ind=8,subact_length = 6, show_fag=False)

    print act_dict.shape,react_dict.shape
    
    
    training_samples, training_ground, test_samples, test_ground = divide_data_set(act_dict, react_dict)
    print 'all:',act_dict.shape[0],'train:',training_samples.shape[0],'test:',test_samples.shape[0]
        
    config_path = os.path.join(CLEANED_FOLDER,class_i+'_skel_pose_'+".npz")
    np.savez(config_path, training_samples=training_samples, training_ground=training_ground,\
             test_samples = test_samples, test_ground = test_ground)
        
    
#     from sklearn import svm
#     X = training_samples
#     y = training_ground
#     print X.shape,y.shape
#     clf = svm.SVC()
#     clf.fit(X,y)
#     print sum(clf.predict(test_samples)==test_ground)/float(len(test_ground))

(734, 480) (734,)
all: 734 train: 587 test: 147
(520, 480) (520,)
all: 520 train: 416 test: 104
(607, 480) (607,)
all: 607 train: 485 test: 122
(895, 480) (895,)
all: 895 train: 716 test: 179
(494, 480) (494,)
all: 494 train: 395 test: 99
(517, 480) (517,)
all: 517 train: 413 test: 104
(750, 480) (750,)
all: 750 train: 600 test: 150
(613, 480) (613,)
all: 613 train: 490 test: 123


In [84]:
# this part is to compare the different classification method on the data
def KNN_test(training_samples, training_ground, test_samples, test_ground):

    from sklearn.neighbors import KDTree
    kdt = KDTree(training_samples, leaf_size=50, metric='euclidean')

    nn_matrix = kdt.query(test_samples, k=15, return_distance=False)

    test_results = np.zeros(test_samples.shape[0]).astype('int')
    for act_i in range(test_samples.shape[0]):
        nn_indices = nn_matrix[act_i]
        nn_values = training_ground[nn_indices]
        counts = np.bincount(nn_values)
        test_results[act_i] = np.argmax(counts)
    
    return sum(test_results==test_ground)/float(test_results.shape[0])
    
    count_error = 0
    for i in range(test_results.shape[0]):
        if test_results[i]>0 and test_ground[i]>0 and test_results[i]<>test_ground[i]:
            count_error = count_error+1

def DT_test(training_samples, training_ground, test_samples, test_ground):
    from sklearn import tree
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(training_samples, training_ground)
    test_results = clf.predict(test_samples)
    #print test_results
    return sum(test_results==test_ground)/float(test_results.shape[0])


for class_i in classes:
    config_path = os.path.join(CLEANED_FOLDER,class_i+".npz")
    data = np.load(config_path)
    training_samples = data['training_samples']
    training_ground = data['training_ground']
    test_samples = data['test_samples']
    test_ground = data['test_ground']
    print "KNN:{0:.2f}\t".format(KNN_test(training_samples, training_ground, test_samples, test_ground)),
    print "Random:{0:.2f}\t".format(random_precision(training_ground)),
    print "DT:{0:.2f}\t".format(DT_test(training_samples, training_ground, test_samples, test_ground)) 

KNN:0.99	Random:0.95	DT:0.98	
KNN:0.98	Random:0.98	DT:0.96	
KNN:0.59	Random:0.43	DT:0.65	
KNN:0.69	Random:0.43	DT:0.70	
KNN:0.59	Random:0.43	DT:0.62	
KNN:0.59	Random:0.37	DT:0.61	
KNN:0.65	Random:0.37	DT:0.67	
KNN:0.63	Random:0.48	DT:0.68	


In [ ]:

tt_proportion = []
for i in np.unique(training_ground):
    proportion = sum(training_ground==i)/float(len(training_ground))
    tt_proportion.extend([proportion])
print tt_proportion
print max(tt_proportion)


torso distance velocity feature:
KNN:0.99	Random:0.95	DT:0.97	
KNN:0.98	Random:0.98	DT:0.98	
KNN:0.59	Random:0.43	DT:0.66	
KNN:0.69	Random:0.43	DT:0.65	
KNN:0.59	Random:0.43	DT:0.58	
KNN:0.59	Random:0.37	DT:0.61	
KNN:0.65	Random:0.37	DT:0.70	
KNN:0.63	Random:0.48	DT:0.63	

+skel to react torso distance feature:
KNN:0.99	Random:0.95	DT:0.97	
KNN:0.98	Random:0.98	DT:0.98	
KNN:0.59	Random:0.43	DT:0.67	
KNN:0.69	Random:0.43	DT:0.65	
KNN:0.59	Random:0.43	DT:0.62	
KNN:0.59	Random:0.37	DT:0.59	
KNN:0.65	Random:0.37	DT:0.68	
KNN:0.63	Random:0.48	DT:0.63	

+skel to its own torso distance feature:
KNN:0.99	Random:0.95	DT:0.98	
KNN:0.98	Random:0.98	DT:0.98	
KNN:0.59	Random:0.43	DT:0.66	
KNN:0.69	Random:0.43	DT:0.66	
KNN:0.59	Random:0.43	DT:0.62	
KNN:0.59	Random:0.37	DT:0.66	
KNN:0.65	Random:0.37	DT:0.69	
KNN:0.63	Random:0.48	DT:0.65	

In [ ]:
a = [1,3]
print a
a.extend([2])
print a

In [ ]:
# visualization learned intense
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
# Attaching 3D axis to the figure
# main function
fn = '/Users/rmqlife/Desktop/CoRL2017/sbu_cleaned/05/s01s02001.txt'


def draw_vector(ax,s,e):
    subtract = s-e
    vlength = np.dot(subtract,subtract)**0.5
    print s, e, s-e
    print s[2],s[0],s[1]

    ax.quiver(s[2],s[0],s[1],e[2],e[0],e[1],length = vlength)
    #ax.quiver(start_point[0],start_point[1],start_point[2],end_point[0],end_point[1],end_point[2],length = vlength)

def canvas():
    fig = plt.figure()
    ax = p3.Axes3D(fig)
    # Setting the axes properties
    ax.set_xlim3d([0.0, 1.0])
    ax.set_xlabel('X')
    ax.set_ylim3d([0.0, 1.0])
    ax.set_ylabel('Y')
    ax.set_zlim3d([1.0, 0.0])
    ax.set_zlabel('Z')
    ax.set_title('3D Test')
    return fig,ax
    
def draw_points(ax,points):
    ax.scatter(points[:,2],points[:,0],points[:,1])

with open(fn,'r') as src_file:
    video = src_file.read().splitlines()
    
    act_subject = get_act_subject(video)
        
    fig, ax = canvas()
    
    #ax.scatter(j1z,j1x,j1y)
    skel_act = get_skels_by_subject(video,act_subject)
    joints_act = get_joints_by_subject(video,act_subject,[5,8])  
    joints_react = get_joints_by_subject(video,1-act_subject,[5,8])
    

    # draw_vector(ax,joints_react[0,:],joints_react[-2,:])    
    draw_points(ax, joints_act)
    draw_points(ax, joints_react)
    plt.show()


In [ ]:
# generate a trajectory

print 1+2